In [1]:
import sys;sys.path.append('..')
from ppanlp.corpus import *
from ppanlp.imports import *

In [4]:
ppa = PPA('~/ppa_data/corpus2')

In [5]:
!pip install peewee

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.3 MB/s eta 0:00:0000:0100:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peewee: filename=peewee-3.17.0-cp310-cp310-macosx_13_0_x86_64.whl size=262452 sha256=4d810c6937e9ff934f4a2703f86520b3401348022297363a61638b4e91042595
  Stored in directory: /Users/ryanheuser/Library/Caches/pip/wheels/c7/70/ad/212867e96e7004265a69c4aa5dcff00a95f547a67ba26e7e76
Successfully built peewee


In [47]:
from peewee import SqliteDatabase, Model, CharField, TextField, IntegerField, FloatField
import datetime

db = SqliteDatabase('my_database5.db')

class BaseModel(Model):
    class Meta:
        database = db

class Page(BaseModel):
    page_id = CharField()
    page_text = TextField()
    page_num_content_words = IntegerField()
    work_id = CharField()
    cluster = CharField()
    source = CharField()
    year = IntegerField()
    author = CharField()
    title = CharField()
    _random = FloatField()

db.connect()

if not db.table_exists(Page): db.create_tables([Page])

In [48]:
def gen(self, force=False):
    if force or (count:=Page.select().where(Page.work_id==self.id).count()) != self.num_pages:
        if force or count: 
            Page.delete().where(Page.work_id==self.id).execute()

        inp = [
            dict(
                page_id=page.id,
                page_text=page.txt,
                page_num_content_words=page.num_content_words,
                work_id=page.text.id,
                cluster = page.text.cluster,
                source = page.text.cluster,
                year = page.text.cluster,
                author = page.text.author,
                title = page.text.title[:255],
                _random = random.random()
            )
            for page in self.pages_json_preproc
        ]
        if inp: Page.insert(inp).execute()

In [49]:
t=ppa.text

In [50]:
gen(t)

In [71]:
def iter_pages(self, work_ids=None, clean=None, lim=None, min_doc_len=None, frac=None, max_per_cluster=None, as_dict=True):
    if not frac or frac>1 or frac<=0: frac=1
    i=0
    clustd=Counter()
    work_ids=set(work_ids) if work_ids else None
    
    q=None
    if frac and min_doc_len:
        q = (Page._random<=frac) & (Page.min_doc_len>=min_doc_len)
    elif frac:
        q = (Page._random<=frac)
    elif min_doc_len:
        q = (Page.min_doc_len>=min_doc_len)
    
    iterr = Page.select().where(q) if q else Page.select()
    
    for page_rec in tqdm(iterr,desc='Iterating over page search results'):
        d=page_rec.__data__        
        if work_ids and d['work_id'] not in work_ids: continue
        if not max_per_cluster or clustd[d['cluster']]<max_per_cluster:
            yield PPAPage(d['page_id'], self.textd[d['work_id']], **d) if not as_dict else d
            if max_per_cluster: clustd[d['cluster']]+=1
            i+=1
            if lim and i>=lim: break
        if lim and i>=lim: break

In [73]:
pd.DataFrame(iter_pages(ppa, lim=10))

Iterating over page search results:   1%|          | 9/1642 [00:00<00:00, 140853.49it/s]


,id,page_id,page_text,page_num_content_words,work_id,cluster,source,year,author,title,_random
0,1,njp.32101080458316_1,\n,0,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.002575
1,2,njp.32101080458316_2,\n,0,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.725910
2,3,njp.32101080458316_3,,0,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.249469
3,4,njp.32101080458316_4,,0,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.054563
4,5,njp.32101080458316_5,,0,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.771075
5,6,njp.32101080458316_6,,0,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.866113
6,7,njp.32101080458316_i,DICTIONARY OF HYMNOLOGY\nSETTING FORTH THE\nOR...,46,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.834579
7,8,njp.32101080458316_ii,"Copyright, 1891, bt\nCHARLES SCHIBNER'S SONS\n...",6,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.018973
8,9,njp.32101080458316_iii,TABLE OF CONTENTS.\nnam\nT. Preface .............,47,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.780394
9,10,njp.32101080458316_iv,,0,njp.32101080458316,juliandictionary,juliandictionary,juliandictionary,"Julian, John, 1839-1913, ed","A dictionary of hymnology,",0.274707


In [68]:
rec.__data__

{'id': 1,
 'page_id': 'njp.32101080458316_1',
 'page_text': '\n',
 'page_num_content_words': 0,
 'work_id': 'njp.32101080458316',
 'cluster': 'juliandictionary',
 'source': 'juliandictionary',
 'year': 'juliandictionary',
 'author': 'Julian, John, 1839-1913, ed',
 'title': 'A dictionary of hymnology,',
 '_random': 0.0025753984256250417}

In [8]:
CharField?

Init signature: CharField(max_length=255, *args, **kwargs)
Docstring:      <no docstring>
File:           ~/github/ppa-nlp/venv/lib/python3.10/site-packages/peewee.py
Type:           type
Subclasses:     FixedCharField

In [3]:
ppa.pages_df(frac=.01, min_doc_len=100).page_num_content_words.describe()

Iterating over page search results: 100%|██████████| 859/859 [00:11<00:00, 77.93it/s] 


count     859.000000
mean      186.233993
std       130.342144
min         0.000000
25%       126.000000
50%       151.000000
75%       192.000000
max      1095.000000
Name: page_num_content_words, dtype: float64

In [4]:
df=ppa.pages_df(frac=.01, min_doc_len=100)
df[df.page_num_content_words==0]

,page_id,page_num,page_num_orig,page_text,page_text_orig,page_tokens,page_corrections_headers,page_corrections_linebreaks,page_corrections_long_s,page_corrections_ocr,page_corrections_f_s,page_content_words,page_num_tokens,page_num_content_words,work_id,cluster,source,year,author,title
140,mdp.39015024941562_10,772,10,\n,\n,[],[],[],[],[],[],[],0,0,mdp.39015024941562,mdp.39015024941562,mdp.39015024941562,1900.0,,Pamphlets in philology and the humanities.
144,mdp.39015024941562_58,738,58,,,[],[],[],[],[],[],[],0,0,mdp.39015024941562,mdp.39015024941562,mdp.39015024941562,1900.0,,Pamphlets in philology and the humanities.
488,hvd.32044086668365_12,282,12,,,[],[],[],[],[],[],[],0,0,hvd.32044086668365,hvd.32044086668365,hvd.32044086668365,1901.0,"Dahlstedt, August, 1867-",Rhythm and word-order in Anglo-Saxon and Semi-...


In [8]:
ppa.gen(num_proc=1)

Saving cleaned pages into page db [x1]:   1%|          | 74/6939 [02:59<5:04:05,  2.66s/it]

In [ ]:
# def iter_pages(self, work_ids=None, clean=None, lim=None, min_doc_len=None, frac=1, frac_text=1, max_per_cluster=None, as_dict=True):
#     if not frac or frac>1 or frac<=0: frac=1
#     # if not frac_text or frac_text>1 or frac_text<=0: frac_text=1
#     ppa.textd
#     i=0
#     clustd=Counter()
#     q={}
#     if frac and frac>0 and frac<1: q['_random']={'$lte':frac}
#     if work_ids: q['work_id']={'$in':list(work_ids)}
#     if min_doc_len: q['page_num_content_words']={'$gte':min_doc_len}
#     for d in tqdm(self.page_db.find(q),desc='Iterating over page search results'):
#         del d['_id']
#         t=self.textd[d['work_id']]
#         if not max_per_cluster or clustd[t.cluster]<max_per_cluster:
#             yield PPAPage(d['page_id'], t, **d) if not as_dict else d
#             if max_per_cluster: clustd[t.cluster]+=1
#             i+=1
#             if lim and i>=lim: break
#         if lim and i>=lim: break


In [ ]:
# f=iter_pages(ppa, max_per_cluster=100, min_doc_len=25, as_dict=True,frac=.1)
# df=pd.DataFrame(f)
# df